In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import seaborn as sns

# 2014 expenditure the US spent on foreign aid. 
# Found here: https://explorer.usaid.gov/query
pd.read_csv("data/us_aid.csv")


,id,country_id,country_code,country_name,region_id,region_name,income_group_id,income_group_name,income_group_acronym,implementing_agency_id,...,transaction_type_id,transaction_type_name,fiscal_year,current_amount,constant_amount,funding_agency_id,funding_agency_name,funding_agency_acronym,aid_type_group_id,aid_type_group_name
0,103666,1002,WLD,World,7,World,0,NaN,NaN,1,...,2,Obligations,2014,1618095498,1594231553,2,Department of State,STATE,2,Core contributions
1,103679,1002,WLD,World,7,World,0,NaN,NaN,4,...,2,Obligations,2014,1355000000,1335016232,28,Executive Office of the President,EOP,2,Core contributions
2,104093,1002,WLD,World,7,World,0,NaN,NaN,6,...,2,Obligations,2014,375830000,370287195,6,Department of Health and Human Services,HHS,3,Project-Type
3,104138,1002,WLD,World,7,World,0,NaN,NaN,1,...,2,Obligations,2014,350000000,344838141,1,U.S. Agency for International Development,USAID,3,Project-Type
4,104342,894,ZMB,Zambia,5,Sub-Saharan Africa,45,Lower Middle Income Country,LMIC,3,...,2,Obligations,2014,272879068,268854601,3,Millennium Challenge Corporation,MCC,3,Project-Type
5,104343,400,JOR,Jordan,3,Middle East and North Africa,44,Upper Middle Income Country,UMIC,1,...,2,Obligations,2014,272500000,268481124,1,U.S. Agency for International Development,USAID,3,Project-Type
6,104488,1002,WLD,World,7,World,0,NaN,NaN,4,...,2,Obligations,2014,226957000,223609800,28,Executive Office of the President,EOP,2,Core contributions
7,104570,1002,WLD,World,7,World,0,NaN,NaN,4,...,2,Obligations,2014,205546750,202515311,4,Department of the Treasury,TREAS,2,Core contributions
8,104686,566,NGA,Nigeria,5,Sub-Saharan Africa,45,Lower Middle Income Country,LMIC,6,...,2,Obligations,2014,188888230,186102474,2,Department of State,STATE,3,Project-Type
9,104755,4,AFG,Afghanistan,4,South and Central Asia,46,Low Income Country,LIC,2,...,2,Obligations,2014,180000000,177345329,2,Department of State,STATE,3,Project-Type
